In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras import layers

# Load dataset from CSV
df = pd.read_csv('ask_fm_final.csv')



print(df.head())

                                                post  author_role
0                        Have you ever been in love?            4
1   Hey michael has his phone back so you can unb...            4
2   Do you think that there are any topics that t...            4
3                                      Who ru dating            4
4   Well your the true bitch no it not a joke whe...            0


In [ ]:
df.dropna(inplace=True)

In [ ]:
print(df.shape)

(61240, 2)


In [ ]:
df['author_role'].dtype

dtype('int64')

In [ ]:
df = df[df['author_role'].isin([0,1])]

In [ ]:
df['author_role'].value_counts()

author_role
0    2546
1      73
Name: count, dtype: int64

In [ ]:
print(df.shape)

(2619, 2)


In [ ]:
for a in df['post']:
  if type(a) != str:
    print('uh oh')

In [ ]:
# Convert the 'post' column to string dtype
df['post'] = df['post'].astype(str)


In [ ]:
print(df['post'].tolist())

[' Well your the true bitch no it not a joke when someone cuts but why would you cut in the first place bitch', ' Well i would still beat up michael', ' I hate you', " I'm leave you fucktard", " i'm a nigga and want you in my bed xxx", ' cuz I want to bring her into it and no bet ya did kill her.', ' Why not? it will be fun lets try. Your sisters dead yh? that must be well guttin for you? what did you kill her or summin? sickoo killing your own sister shes better off dead anyway with a sister like you', ' Your nothing but scum m8', ' so you get all your little friends to stick up for you? cant fight your own battles then? I only said what I think you should do and yeah I got my sister it would kill me if I lost her but shes not the one whos dead its your sister not mine! get the fuck over it you will never see her again:)', " just go die already ye? you have to realise that nobody wants you here your better off dead.i don't care if its 'tight' you should be dead the world will be bette

In [ ]:
print(type(df['post'].tolist()[0]))
print(df['post'].tolist()[0])

<class 'str'>
 Well your the true bitch no it not a joke when someone cuts but why would you cut in the first place bitch


In [ ]:
print(type(df['post'].tolist()))

<class 'list'>


In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras import layers


# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define your model class
class DistillBERTClass(tf.keras.Model):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.bert = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = layers.Dense(768, activation='relu')
        self.classifier = layers.Dense(2)

    def call(self, inputs):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        output = self.bert(input_ids, attention_mask=attention_mask)
        cls_hidden_state = output.last_hidden_state[:, 0, :]
        pre_classify_output = self.pre_classifier(cls_hidden_state)
        return self.classifier(pre_classify_output)

# Tokenize text data
tokenized_texts = tokenizer(df['post'].tolist(), padding=True, max_length = 128, truncation=True,return_tensors="tf")

# Define inputs and attention masks
inputs = {
    'input_ids': tokenized_texts['input_ids'],
    'attention_mask': tokenized_texts['attention_mask']
}




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
y_train_one_hot = tf.keras.utils.to_categorical(df['author_role'], num_classes=2)
print(y_train_one_hot[:5])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
from tensorflow.keras import metrics


# Define your metrics
metrics = [
   metrics.CategoricalAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall'),
    metrics.F1Score(name='f1_score')
]

# Compile model
model = DistillBERTClass()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[metrics])

# Train model
model.fit(inputs, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/10
66/66 [==============================] - 54s 468ms/step - loss: 0.1243 - accuracy: 0.9737 - precision: 0.9741 - recall: 0.9504 - f1_score: 0.4933 - val_loss: 0.1330 - val_accuracy: 0.9656 - val_precision: 0.9656 - val_recall: 0.9656 - val_f1_score: 0.4913
Epoch 2/10
66/66 [==============================] - 28s 422ms/step - loss: 0.0995 - accuracy: 0.9718 - precision: 0.9783 - recall: 0.9475 - f1_score: 0.4929 - val_loss: 0.1180 - val_accuracy: 0.9656 - val_precision: 0.9724 - val_recall: 0.9427 - val_f1_score: 0.4913
Epoch 3/10
66/66 [==============================] - 27s 414ms/step - loss: 0.0520 - accuracy: 0.9809 - precision: 0.9935 - recall: 0.9504 - f1_score: 0.7570 - val_loss: 0.1378 - val_accuracy: 0.9656 - val_precision: 0.9746 - val_recall: 0.9504 - val_f1_score: 0.5821
Epoch 4/10
66/66 [==============================] - 27s 416ms/step - loss: 0.0273 - accuracy: 0.9905 - precision: 0.9956 - recall: 0.9790 - f1_score: 0.9050 - val_loss: 0.1568 - val_accuracy: 0.9618 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

y_pred = np.argmax(model.predict(inputs), axis=1)

# Convert the true labels to integers
y_true = np.argmax(y_train_one_hot, axis=1)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

82/82 [==============================] - 15s 135ms/step
Confusion Matrix:
[[2542    4]
 [  16   57]]


In [ ]:
#Really good at predicting harrassment
#F1 score improved quite a bit

In [ ]:
# Assuming you provided two new samples
new_samples = ["You are an ugly pig", "Kill yourself","You're hurting my feelings","Leave me alone!","im nice"]

# Tokenize the new samples
new_tokenized_texts = tokenizer(new_samples, padding=True, max_length=128, truncation=True, return_tensors="tf")

# Create inputs and attention masks for the new samples
new_inputs = {
    'input_ids': new_tokenized_texts['input_ids'],
    'attention_mask': new_tokenized_texts['attention_mask']
}

# Predict the classes for the new samples
new_predictions = model.predict(new_inputs)

# Convert predicted probabilities to class labels
predicted_labels = tf.argmax(new_predictions, axis=1).numpy()

# Print the predicted labels
print("Predicted labels:", predicted_labels)


1/1 [==============================] - 0s 55ms/step
Predicted labels: [0 0 0 1 0]
